In [1]:
sc

org.apache.spark.SparkContext@7c5a1d26

In [2]:
val data = Array(1, 2, 3, 4, 5)
data

Array(1, 2, 3, 4, 5)

In [3]:
val distData = sc.parallelize(data)
distData

ParallelCollectionRDD[1] at parallelize at <console>:22

In [4]:
val lines = sc.textFile("input/case.csv")
lines

input/case.csv MapPartitionsRDD[3] at textFile at <console>:20

In [5]:
lines.count()

14046

In [6]:
lines.take(5)

Array(00013D2EFD8E45D1,DIAG78820,1166,1.0, 00013D2EFD8E45D1,DIAGV4501,1166,1.0, 00013D2EFD8E45D1,heartfailure,1166,1.0, 00013D2EFD8E45D1,DIAG2720,1166,1.0, 00013D2EFD8E45D1,DIAG4019,1166,1.0)

In [7]:
lines.take(5).foreach(println)

00013D2EFD8E45D1,DIAG78820,1166,1.0
00013D2EFD8E45D1,DIAGV4501,1166,1.0
00013D2EFD8E45D1,heartfailure,1166,1.0
00013D2EFD8E45D1,DIAG2720,1166,1.0
00013D2EFD8E45D1,DIAG4019,1166,1.0


In [8]:
lines.map(line => line.split(",")(0))

MapPartitionsRDD[4] at map at <console>:23

In [9]:
lines.map{line =>
  val s = line.split(",")
  (s(0), s(1))
}

MapPartitionsRDD[5] at map at <console>:23

In [10]:
lines.filter(line => line.contains("00013D2EFD8E45D1")).count()

200

In [11]:
lines.map(line => line.split(",")(0)).distinct().count()

100

In [12]:
val patientIdEventPair = lines.map{line =>
  val patientId = line.split(",")(0)
  (patientId, line)
}
val groupedPatientData = patientIdEventPair.groupByKey
groupedPatientData.take(1)

Array((0102353632C5E0D0,CompactBuffer(0102353632C5E0D0,DIAG29181,562,1.0, 0102353632C5E0D0,DIAG29212,562,1.0, 0102353632C5E0D0,DIAG34590,562,1.0, 0102353632C5E0D0,DIAG30000,562,1.0, 0102353632C5E0D0,DIAG2920,562,1.0, 0102353632C5E0D0,DIAG412,562,1.0, 0102353632C5E0D0,DIAG28800,562,1.0, 0102353632C5E0D0,DIAG30391,562,1.0, 0102353632C5E0D0,DIAGRG894,562,1.0, 0102353632C5E0D0,PAYMENT,562,6000.0, 0102353632C5E0D0,DIAG5781,570,1.0, 0102353632C5E0D0,DIAG53010,570,1.0, 0102353632C5E0D0,DIAGE8490,570,1.0, 0102353632C5E0D0,DIAG27651,570,1.0, 0102353632C5E0D0,DIAG78559,570,1.0, 0102353632C5E0D0,DIAG56210,570,1.0, 0102353632C5E0D0,DIAG5856,570,1.0, 0102353632C5E0D0,heartfailure,570,1.0, 0102353632C5E0D0,DIAG5070,570,1.0, 0102353632C5E0D0,DIAGRG346,570,1.0...

In [14]:
val payment_events = lines.filter(line => line.contains("PAYMENT"))
val payments = payment_events.map{ x =>
                                   val s = x.split(",")
                                   (s(0), s(3).toFloat)
                                 }
val paymentPerPatient = payments.reduceByKey(_+_)

In [15]:
paymentPerPatient.sortBy(_._2, false).take(3).foreach(println)

(0085B4F55FFA358D,139880.0)
(019E4729585EF3DD,108980.0)
(01AC552BE839AB2B,108530.0)


In [16]:
val payment_values = paymentPerPatient.map(payment => payment._2).cache()

In [17]:
payment_values.max()

139880.0

In [18]:
payment_values.min()

3910.0

In [21]:
payment_values.sum()

2842480.0

In [19]:
payment_values.mean()

28424.800000000003

In [20]:
 payment_values.stdev()

26337.091771112464

In [22]:
val linesControl = sc.textFile("input/control.csv")
lines.union(linesControl).count() 

31144